In [1]:
import sys
sys.path.append("../cs224u")
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from collections import Counter
import vsm

In [2]:
analysts = pd.read_csv('./data/analysts.csv')
print(analysts.shape)
analysts.head(10)

(187327, 12)


,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,analyst-company
0,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",Bruce Geller - DGHM
1,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",John Healy - Northcoast Research
2,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",Kevin Steinke - Barrington Research Associates...
3,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",Joe Box - KeyBanc Capital Markets
4,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET","Justin Hauke - Robert W. Baird & Company, Inc."
5,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",Andrew Steinerman - JPMorgan
6,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",Kevin McVeigh - Deutsche Bank
7,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",Mark Marcon - Baird
8,4,Sonic Corp.,NASDAQ,SONC,2017-01-04 21:41:07,Sonic's (SONC) CEO Cliff Hudson on Q1 2017 Res...,Sonic Corp.,Q1 2017 Results Earnings Conference Call,NaN,SONC,"January 04, 2017 05:00 PM ET",Greg Francfort - Bank of America
9,4,Sonic Corp.,NASDAQ,SONC,2017-01-04 21:41:07,Sonic's (SONC) CEO Cliff Hudson on Q1 2017 Res...,Sonic Corp.,Q1 2017 Results Earnings Conference Call,NaN,SONC,"January 04, 2017 05:00 PM ET",Nicole Miller - Piper Jaffray


In [3]:
execs = pd.read_csv('./data/executives.csv')
print(execs.shape)
execs.head(10)

(111916, 12)


,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,executive-positions
0,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",Bob Stewart - President and Chief Financial Of...
1,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",Michael Weinstein - Chairman and Chief Executi...
2,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",Ronald Croatti - President and CEO
3,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",Steven Sintros - SVP and CFO
4,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",Alice Washington - Interim General Counsel
5,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",Kate Duchene - Chief Executive Officer
6,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",Herb Mueller - Chief Financial Officer
7,4,Sonic Corp.,NASDAQ,SONC,2017-01-04 21:41:07,Sonic's (SONC) CEO Cliff Hudson on Q1 2017 Res...,Sonic Corp.,Q1 2017 Results Earnings Conference Call,NaN,SONC,"January 04, 2017 05:00 PM ET",Cliff Hudson - CEO
8,4,Sonic Corp.,NASDAQ,SONC,2017-01-04 21:41:07,Sonic's (SONC) CEO Cliff Hudson on Q1 2017 Res...,Sonic Corp.,Q1 2017 Results Earnings Conference Call,NaN,SONC,"January 04, 2017 05:00 PM ET",Claudia San Pedro - CFO
9,5,"Walgreens Boots Alliance, Inc.",NASDAQ,WBA,2017-01-05 16:33:05,Walgreens Boots Alliance's (WBA) CEO Stefano P...,"Walgreens Boots Alliance, Inc.",Q1 2017 Results Earnings Conference Call,NaN,WBA,"January 05, 2017, 08:30 AM ET","Gerald Gradwell - Senior Vice President, Inves..."


In [7]:
qna = pd.read_csv('./data/qna.csv')
print(qna.shape)
qna.head(10)

(1828257, 14)


,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,order,name,content
0,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",1,Operator,[Operator Instructions] Our first question com...
1,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",2,Bruce Geller,Hi Good morning guys.
2,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",3,Bob Stewart,"Hi Bruce, how are you?"
3,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",4,Bruce Geller,Good thanks. Glad to hear things have turned a...
4,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",5,Bob Stewart,"Yes, that includes an additional week last yea..."
5,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",6,Bruce Geller,So it's not nearly as...
6,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",7,Bob Stewart,"No, no."
7,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",8,Bruce Howard Geller,…as dismal as it might imply. What were the ap...
8,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",9,Bob Stewart,"About 2%. Probably down, somewhere along that ..."
9,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",10,Bruce Geller,Got it. Okay. Can you just kind of go through ...


In [6]:
statements = pd.read_csv('./data/statements.csv')
print(statements.shape)
statements.head(10)

(204656, 14)


,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,order,name,content
0,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",1,Presentation,NaN
1,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",2,Operator,"Greetings, and welcome to the Ark Restaurants ..."
2,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",3,Bob Stewart,"Thank you, operator. Good morning, and thank y..."
3,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",4,Michael Weinstein,"Hi, everybody. If you read the release, we're ..."
4,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",1,Presentation,NaN
5,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",2,Operator,"Ladies and gentlemen, thank you for standing b..."
6,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",3,Steven Sintros,Thank you and welcome to the UniFirst Corporat...
7,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",4,Ronald Croatti,"Thank you, Steve. I'd like to welcome everyone..."
8,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",5,Steven Sintros,"Thanks, Ron. As Ron mentioned, revenues for th..."
9,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",1,Presentation,NaN


In [4]:
# Remove earnings calls that do not include a Q&A session

full_statements = pd.DataFrame(columns = cols)
full_statements.head()

,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,order,name,content


In [ ]:
row_list = []
for i in range(statements.shape[0]):
    publish_date = statements.iloc[i]['file-published-on']
    if len(qna.loc[qna['file-published-on'] == publish_date]) != 0: row_list.append(statements.iloc[i])

cols = ['num', 'file-company-name', 'file-exchange', 'file-symbol', 'file-published-on', 'file-title', 'company-name', 'quarter-year-event', 'symbol-exchange', 'symbol', 'datetime', 'order', 'name', 'content']
comp_statements = pd.DataFrame(row_list,columns = cols)
print(comp_statements.shape)
comp_statements.head(10)

In [ ]:
row_list = []
for i in range(execs.shape[0]):
    publish_date = execs.iloc[i]['file-published-on']
    if len(qna.loc[qna['file-published-on'] == publish_date]) != 0: row_list.append(execs.iloc[i])

cols = ['num', 'file-company-name', 'file-exchange', 'file-symbol', 'file-published-on', 'file-title', 'company-name', 'quarter-year-event', 'symbol-exchange', 'symbol', 'datetime', 'executive-positions']
comp_execs = pd.DataFrame(row_list,columns = cols)
print(comp_execs.shape)
comp_execs.head(10)